In [4]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [5]:
url = 'https://www.wikidata.org/wiki/Wikidata:WikiProject_Freebase/Mapping'

In [6]:
html_txt = requests.get(url)

In [7]:
soup = BeautifulSoup(html_txt.text)

In [8]:
tables = soup.find_all('table')

In [9]:
urls = []

for table in tables:
    for e in table.find_all('tr'):
        res = []
        for a in e.find_all('a'):
            res.append(a['href'])

        if len(res) == 2:
            urls.append(res)

In [10]:
urls_dict = {}

for freebase, wikidata in urls:
    if 'freebase' in freebase and 'Property' in wikidata:
        prop = wikidata.split(':')[-1]
        freebase = freebase.replace('https://', '')
        
        urls_dict[prop] = freebase

In [11]:
urls_dict

{'P131': 'www.freebase.com/location/hud_county_place/county',
 'P150': 'www.freebase.com/location/country/first_level_divisions',
 'P413': 'www.freebase.com/sports/sports_team_roster/position',
 'P800': 'www.freebase.com/book/author/works_written',
 'P84': 'www.freebase.com/architecture/structure/architect',
 'P149': 'www.freebase.com/architecture/structure/architectural_style',
 'P1619': 'www.freebase.com/architecture/structure/opened',
 'P196': 'www.freebase.com/astronomy/asteroid/member_of_asteroid_group',
 'P61': 'www.freebase.com/chemistry/element_discoverer/discovered',
 'P575': 'www.freebase.com/astronomy/astronomical_discovery/discovery_date',
 'P65': 'www.freebase.com/astronomy/astronomical_discovery/discovery_site',
 'P31': 'www.freebase.com/transportation/bridge/bridge_type',
 'P279': 'www.freebase.com/digicams/digital_camera/format',
 'P376': 'www.freebase.com/astronomy/extraterrestrial_location/on_celestial_object',
 'P398': 'www.freebase.com/astronomy/orbital_relationship

In [13]:
triplets = pd.read_json('data/poet_triplets.json')
triplets.head()

,source,edge,destination
0,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P31', 'name': 'osztály, amely...","{'wikidata_id': 'Q5', 'name': 'ember', 'descri..."
1,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P21', 'name': 'nem', 'descrip...","{'wikidata_id': 'Q6581097', 'name': 'férfi', '..."
2,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P735', 'name': 'utónév', 'des...","{'wikidata_id': 'Q17498051', 'name': 'József',..."
3,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P570', 'name': 'halálozási id...","{'time': '+1988-02-02T00:00:00Z', 'timezone': ..."
4,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P106', 'name': 'foglalkozás',...","{'wikidata_id': 'Q49757', 'name': 'költő', 'de..."


In [14]:
properties = triplets.edge.apply(lambda d: d['wikidata_id']).tolist()

In [15]:
urls_dict_filt = {k: v for k, v in urls_dict.items() if k in properties}

In [16]:
urls_dict_filt

{'P800': 'www.freebase.com/book/author/works_written',
 'P31': 'www.freebase.com/transportation/bridge/bridge_type',
 'P166': 'www.freebase.com/award/award_winning_work/awards_won',
 'P1411': 'www.freebase.com/award/award_nominee/award_nominations',
 'P460': 'www.freebase.com/base/givennames/given_name/variations',
 'P123': 'www.freebase.com/cvg/computer_videogame/publisher',
 'P136': 'http://www.freebase.com/authority/netflix/genre',
 'P856': 'www.freebase.com/music/artist/home_page',
 'P512': 'www.freebase.com/education/education/degree',
 'P69': 'www.freebase.com/people/person/education',
 'P39': 'www.freebase.com/government/politician/government_positions_held',
 'P102': 'www.freebase.com/government/politician/party',
 'P345': 'http://www.freebase.com/authority/imdb/title',
 'P276': 'http://www.freebase.com/transportation/bridge/locale',
 'P607': 'www.freebase.com/military/military_person/participated_in_conflicts',
 'P241': 'www.freebase.com/military/military_service/military_forc

In [17]:
triplets.edge.apply(lambda d: 1 if d['wikidata_id'] in urls_dict_filt else 0).sum()

21706

In [18]:
len(triplets)

41281

In [19]:
question_df = pd.read_csv('https://raw.githubusercontent.com/davidgolub/SimpleQA/master/datasets/SimpleQuestions/annotated_fb_data_train.txt', sep='\t', header=None)

In [20]:
question_df.columns = ['e1', 'property', 'e2', 'question']

In [21]:
filt_question = question_df[question_df.property.isin(list(urls_dict_filt.values()))]
len(filt_question)

12140

In [31]:
filt_question = filt_question.drop_duplicates(subset=['question'])
len(filt_question)

12136

In [32]:
filt_question.groupby(by='property').question.count().sort_values()

property
www.freebase.com/royalty/monarch/royal_line                               7
www.freebase.com/transportation/bridge/bridge_type                       14
www.freebase.com/sports/sports_team/sport                                37
www.freebase.com/tv/tv_series_episode/season                             57
www.freebase.com/people/deceased_person/place_of_burial                  69
www.freebase.com/music/release/label                                     74
www.freebase.com/military/military_person/participated_in_conflicts     197
www.freebase.com/cvg/computer_videogame/publisher                       200
www.freebase.com/music/group_member/instruments_played                  299
www.freebase.com/people/person/children                                 320
www.freebase.com/book/author/works_written                              321
www.freebase.com/people/person/ethnicity                                355
www.freebase.com/people/person/religion                                 442
www

In [40]:
samp_questions = filt_question.groupby(by='property').sample(n=300, replace=True).drop_duplicates(subset=['question'])

In [41]:
samp_questions.groupby(by='property').question.count().sort_values()

property
www.freebase.com/royalty/monarch/royal_line                              7
www.freebase.com/transportation/bridge/bridge_type                      14
www.freebase.com/sports/sports_team/sport                               37
www.freebase.com/tv/tv_series_episode/season                            57
www.freebase.com/people/deceased_person/place_of_burial                 65
www.freebase.com/music/release/label                                    73
www.freebase.com/cvg/computer_videogame/publisher                      153
www.freebase.com/military/military_person/participated_in_conflicts    154
www.freebase.com/music/group_member/instruments_played                 189
www.freebase.com/people/person/children                                192
www.freebase.com/book/author/works_written                             201
www.freebase.com/people/person/ethnicity                               201
www.freebase.com/people/person/religion                                222
www.freebase.com

In [42]:
samp_questions.question.apply(len).sum()

122085

# Translation

In [1]:
from google.cloud import translate

In [35]:
def translate_text(text="YOUR_TEXT_TO_TRANSLATE", project_id="YOUR_PROJECT_ID"):
    """Translating Text."""

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            "source_language_code": "en-US",
            "target_language_code": "hu",
        }
    )

    # Display the translation for each input text provided
    for translation in response.translations:
        return translation.translated_text


In [36]:
translate_text('hi', 'infinite-badge-193208')

'Szia'

In [44]:
samp_questions['hun_question'] = samp_questions.question.apply(lambda s: translate_text(s, 'infinite-badge-193208'))

In [45]:
samp_questions

,e1,property,e2,question,hun_question
50252,www.freebase.com/m/01xdf5,www.freebase.com/book/author/works_written,www.freebase.com/m/02qxck5,What book did stephen colbert write?,Milyen könyvet írt Stephen Colbert?
9759,www.freebase.com/m/03m9c9l,www.freebase.com/book/author/works_written,www.freebase.com/m/04wbprh,whats the name of michael mcgarrity's books,mi a neve Michael McCgarrity könyveinek
16708,www.freebase.com/m/041h0,www.freebase.com/book/author/works_written,www.freebase.com/m/04t05ry,what was written by j. r. r. tolkien,amit j írt. r. r. tolkien
59961,www.freebase.com/m/031qmr,www.freebase.com/book/author/works_written,www.freebase.com/m/04w95tk,what was one of pete hamill's works,mi volt Pete Hamill egyik műve
46516,www.freebase.com/m/01st1f,www.freebase.com/book/author/works_written,www.freebase.com/m/05qcvw1,What book did andre norton write?,Milyen könyvet írt andre norton?
...,...,...,...,...,...
26424,www.freebase.com/m/0jsnq3x,www.freebase.com/tv/tv_series_episode/season,www.freebase.com/m/0k2czv4,what season is amsterdam from,milyen évszakból van Amszterdam
4840,www.freebase.com/m/05nfl17,www.freebase.com/tv/tv_series_episode/season,www.freebase.com/m/05ngc1y,what show and season did ben stiller appear in,milyen műsorban és évadban jelent meg ben stiller
1207,www.freebase.com/m/0jwgww8,www.freebase.com/tv/tv_series_episode/season,www.freebase.com/m/0k2d4dj,which tv show features an episode called tom c...,melyik tévéműsorban szerepel egy Tom Cruise ne...
45459,www.freebase.com/m/0jgxnlv,www.freebase.com/tv/tv_series_episode/season,www.freebase.com/m/0k2cp7m,Which season of chelsea lately was paul rudd on,A chelsea melyik szezonja volt az utóbbi időbe...


In [46]:
samp_questions.to_json('data/freebase_samp_questions.json', orient='records')